In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
import datasets
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizerFast, LlamaForCausalLM, LlamaTokenizer
from sklearn.metrics import classification_report
from peft import PeftModel

from eval_utils import *

df_test = pd.read_excel("data/multipleChoice.xlsx" )
# only keep first 50 rows
df_test


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/sebastianquintero/Library/CloudStorage/OneDrive-QuinteroOrthodontics/MIT/MEng/rag-thesis/.venv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/sebastianquintero/Library/CloudStorage/OneDrive-QuinteroOrthodontics/MIT/MEng/rag-thesis/.venv/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    ap

,Question,OptionA,OptionB,OptionC,OptionD,Answer,Feedback,theory,numerical,grouping,fill_in_blank,true_false
0,One of the reasons protectionists and governme...,quotas generate more revenue for the governmen...,quotas ensure that the quantities of imports a...,quotas create less market distortions than tar...,quotas give less power to politicians than tar...,B,NaN,1,0,0,0,0
1,"In the case of a small country, the effects of...",the government allocates licenses for free to ...,the government auctions off import licenses to...,the government allocates licenses to importers...,the government allocates import licenses direc...,B,NaN,0,1,0,0,0
2,A small country imports T-shirts. With free tr...,gain $5 million.,lose $5 million.,gain $25 million.,gain $30 million.,C,NaN,0,1,0,0,0
3,A small country imports T-shirts. With free tr...,gain $7 million.,lose $7 million.,lose $70 million.,lose $77 million.,D,NaN,0,1,0,0,0
4,A small country imports T-shirts. With free tr...,$30 million,$40 million,$70 million,$240 million,B,NaN,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
218,Specific factors are more likely to favor trad...,True,False,NaN,NaN,A,Workers can move away from the sector whose pr...,0,0,0,0,1
219,Factor prices are more likely to be equalized ...,True,False,NaN,NaN,B,"When countries are not completely specialized,...",0,0,0,0,1
220,The European Common Agricultural Policy is a b...,True,False,NaN,NaN,B,The European Common Agricultural Policy entail...,0,0,0,0,1
221,Immigration necessarily lowers wages.,True,False,NaN,NaN,B,"For example, when factor price equalization ho...",0,0,0,0,1


In [2]:
# Sum up the dummy variables to get counts for each category
category_counts = df_test[['theory', 'numerical', 'grouping', 'fill_in_blank', 'true_false']].sum()

# Calculate percentages
total_questions = len(df_test)
category_percentages = (category_counts / total_questions * 100).round(2)

# Create a summary DataFrame
summary_df = pd.DataFrame({
    'Count': category_counts,
    'Percentage (%)': category_percentages
})

print("Question Type Distribution:")
print("==========================")
print(summary_df)
print(f"\nTotal number of questions: {total_questions}")

Question Type Distribution:
               Count  Percentage (%)
theory            36           16.14
numerical         11            4.93
grouping           5            2.24
fill_in_blank     82           36.77
true_false        89           39.91

Total number of questions: 223


In [3]:
cot_prompt = """Lets think step by step in the folliwng multiple choice question:

Question: The country Rich is relatively well endowed with skilled labor whereas its trade partner, Poor, is relatively well endowed with unskilled labor. The two countries produce and freely trade two goods using the same constant-returns-to-scale technologies. The countries have identical and homothetic preferences. In this setting, when trade opens, what happens?

A) The real wage of skilled workers in Rich must rise, the real wage of unskilled
workers in Rich must fall, and the income rise for skilled workers need not exceed
the income fall for unskilled workers.

B) The real wage of unskilled workers in Rich must rise, the real wage of skilled
workers in Rich must fall, and the income rise for unskilled workers need not exceed the income fall for skilled workers

C) The real wage of unskilled workers in Rich must rise, the real wage of skilled
workers in Rich must fall, and the income rise for unskilled workers must exceed the income fall for skilled workers.

D) The real wage of skilled workers in Rich must rise, the real wage of unskilled workers in Rich must fall, and the income rise for skilled workers must exceed the income fall for unskilled workers.

Answer: 
1. By the Heckscher-Ohlin Theorem, the relative price of the skill-intensive good must increase in Rich. 
2. By the Stolper-Samuelson theorem, the real wage for skilled workers in Rich must rise and the real wage for unskilled workers in Rich must fall. 
3. Since there are aggregate gains from trade, the income rise for skilled workers as a whole must exceed the income fall for unskilled workers as a whole. 
4. Thus,the answer is option D: "The real wage of skilled workers in Rich must rise, the real wage of unskilled workers in Rich must fall, and the income rise for skilled workers must exceed the income fall for unskilled workers."

Now answer the following multiple choice question with ONLY a single letter (A, B, C, or D). Do not include any other text, punctuation, or explanation - just the letter.

Question: {}

A) {}
B) {}
C) {}
D) {}
Answer:
"""

cot_prompt_tf = """Lets think step by step in the followng true or false question:

Question: Trade between the US and China is more likely to benefit US high-tech workers and hurt US low-tech workers.

A) True
B) False

Answer: 
1. The US is more abundant in high-tech workers than in low-tech workers relative to China. 
2. Trade will induce a decline in the wage of the US low-tech workers relative to the wage of the US high-tech workers, as low-tech labor becomes less scarse in the US afterstarting to trade with China. 
3. Said differently, trade essentially allows a country to convert a good x into another good y; so one can see it as new technology.
4. Thus, the answer is A: True.

Answer the following true/false question. Respond only with the letter A if the statement is true, or B if the statement is false. Do not include any other text, punctuation, or explanation - just the letter.

Question: {}

A) {}
B) {}
Answer:"""


In [6]:
icl_prompt = """Answer each multiple choice question with ONLY a single letter (A, B, C, or D).
Here are some examples:

Example 1: There are two large countries, the United States and China, and two goods, solar panels and soy bean. The United exports soy beans and imports solar panels. If the United States imposes a small import tariff on solar panels, whereas China imposes a small import tariff on soy beans, then:

A) Both countries are better off than under free trade.
B) Both countries are worse off than under free trade.
C) The United States is better off than under free trade, but China is worse off.
D) China is better off than under free trade, but the United States is worse off.

Answer: B) Both countries are worse off than under free trade.

Example 2: Consider a Ricardian trade model with two goods, Wine and Cheese, and many countries. Suppose France requires less labor to produce Cheese than Italy. Then:

A) France might import Cheese from Italy.
B) France might import Cheese from the rest of the world, but cannot import from Italy.
C) France must export Cheese to country Italy.
D) France must export Cheese to the rest of the world, but not necessarily to Italy.

Answer: A) France might import Cheese from Italy.

Example 3: If a tariff decreases domestic consumption of a good from 230 million units to 150 million units and raises the domestic price by $1.50, given a linear domestic demand curve and a perfectly elastic world supply curve, what is the value of the unexploited gains from trade caused by decreased domestic consumption?

A) $45 million
B) $60 million
C) $80 million
D) $120 million

Answer: B) $60 million

Now answer the following multiple choice question with ONLY a single letter (A, B, C, or D). Do not include any other text, punctuation, or explanation - just the letter.

Question: {}

A) {}
B) {}
C) {}
D) {}
Answer:
"""

icl_prompt_tf = """Examine the answers to the following true or false question:

Example 1: Offshoring cannot raise the real wage of the workers whose jobs are being offshored.
A) True
B) False

Answer: B) False

Example 2: Specific factors are more likely to favor trade protection than mobile factors.
A) True
B) False

Answer: A) True


Example 3: Factor prices are more likely to be equalized across countries if countries are completely specialized.
A) True
B) False

Answer: B) False


Answer the following true/false question. Respond only with the letter A if the statement is true, or B if the statement is false. Do not include any other text, punctuation, or explanation - just the letter.

Question: {}

A) {}
B) {}
Answer:"""


### Chain of Thought Prompting

In [4]:
for MODEL_NAME in ["gpt-35-turbo"]:
    y_pred = []
    y_true = []
    
    for row in tqdm(df_test.iterrows(), total=len(df_test)):
        time.sleep(1)
        question = row[1]["Question"]
        choice_a = row[1]["OptionA"]
        choice_b = row[1]["OptionB"]
        choice_c = row[1]["OptionC"]
        choice_d = row[1]["OptionD"]
        label = row[1]["Answer"].strip().upper()  # Clean the true label
        
        if MODEL_NAME == "gpt-35-turbo":
            if row[1]["true_false"]:
                prediction = prompt_chatgpt_with_backoff(cot_prompt_tf.format(
                    question, choice_a, choice_b))
            else:
                prediction = prompt_chatgpt_with_backoff(cot_prompt.format(
                    question, choice_a, choice_b, choice_c, choice_d))
        
        # Clean and extract the predicted answer letter more carefully
        pred_letter = prediction.strip().upper()
        # If the prediction contains more than just the letter, take first word
        if len(pred_letter) > 1:
            pred_letter = pred_letter.split()[0]
        
        # Validate prediction format
        if pred_letter not in ['A', 'B', 'C', 'D']:
            print(f"Warning: Invalid prediction format: '{prediction}' for question: {question}")
            pred_letter = 'INVALID'
        
        y_pred.append(pred_letter)
        y_true.append(label)

        """
        # Print debugging info for all predictions
        if pred_letter != label:
            print("\nIncorrect Prediction:")
            print(f"Question: {question}")
            print(f"Choices:")
            print(f"A) {choice_a}")
            print(f"B) {choice_b}")
            if not row[1]["true_false"]:
                print(f"C) {choice_c}")
                print(f"D) {choice_d}")
            print(f"Raw model response: '{prediction}'")
            print(f"Processed prediction: '{pred_letter}'")
            print(f"Correct answer: '{label}'")
            print("=========================================================")
        """

    # Calculate overall accuracy
    total_questions = len(y_true)
    correct_answers = sum(1 for pred, true in zip(y_pred, y_true) if pred == true)
    overall_accuracy = correct_answers / total_questions

    # Print detailed matching information
    print("\nDetailed Answer Matching:")
    print("------------------------")
    mismatches = [(i, pred, true) for i, (pred, true) in enumerate(zip(y_pred, y_true)) if pred != true]
    print(f"Found {len(mismatches)} mismatches out of {total_questions} questions")
    
    # Calculate per-category accuracies
    categories = ['theory', 'numerical', 'grouping', 'fill_in_blank', 'true_false']
    category_results = {}
    
    for category in categories:
        # Get questions belonging to this category
        category_indices = df_test[df_test[category] == 1].index
        
        if len(category_indices) > 0:
            category_correct = sum(1 for i in category_indices if y_pred[i] == y_true[i])
            category_accuracy = category_correct / len(category_indices)
            category_results[category] = {
                'accuracy': category_accuracy,
                'correct': category_correct,
                'total': len(category_indices)
            }

    # Print results
    print(f"\nResults for {MODEL_NAME}: CoT Prompting")
    print("=" * 50)
    print(f"Overall Accuracy: {overall_accuracy:.4f} ({correct_answers}/{total_questions} correct)")
    print("\nPer-Category Performance:")
    print("-" * 30)
    
    for category, results in category_results.items():
        print(f"{category.replace('_', ' ').title()}:")
        print(f"  Accuracy: {results['accuracy']:.4f}")
        print(f"  Correct: {results['correct']}/{results['total']}")
        print()

    # Save results to file
    with open("results/LLM_results.txt", "a") as f:
        f.write(f"\nResults for {MODEL_NAME}\n")
        f.write("=" * 50 + "\n")
        f.write(f"Overall Accuracy: {overall_accuracy:.4f} ({correct_answers}/{total_questions} correct)\n")
        f.write("\nPer-Category Performance:\n")
        f.write("-" * 30 + "\n")
        
        for category, results in category_results.items():
            f.write(f"{category.replace('_', ' ').title()}:\n")
            f.write(f"  Accuracy: {results['accuracy']:.4f}\n")
            f.write(f"  Correct: {results['correct']}/{results['total']}\n\n")
        f.write("\n")

 83%|████████▎ | 184/223 [2:22:32<15:59, 24.60s/it]  

Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-12-01-preview have exceeded call rate limit of your current AIServices S0 pricing tier. Please retry after 8 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit. For Free Account customers, upgrade to Pay as you Go here: https://aka.ms/429TrialUpgrade.'}}


 83%|████████▎ | 186/223 [2:23:17<13:48, 22.40s/it]

Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-12-01-preview have exceeded call rate limit of your current AIServices S0 pricing tier. Please retry after 4 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit. For Free Account customers, upgrade to Pay as you Go here: https://aka.ms/429TrialUpgrade.'}}


 84%|████████▍ | 187/223 [2:23:54<16:01, 26.70s/it]

...

Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-12-01-preview have exceeded call rate limit of your current AIServices S0 pricing tier. Please retry after 8 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit. For Free Account customers, upgrade to Pay as you Go here: https://aka.ms/429TrialUpgrade.'}}


100%|█████████▉| 222/223 [2:46:22<00:31, 31.42s/it]

Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-12-01-preview have exceeded call rate limit of your current AIServices S0 pricing tier. Please retry after 8 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit. For Free Account customers, upgrade to Pay as you Go here: https://aka.ms/429TrialUpgrade.'}}


100%|██████████| 223/223 [2:46:55<00:00, 44.91s/it]


Detailed Answer Matching:
------------------------
Found 75 mismatches out of 223 questions

Results for gpt-35-turbo: CoT Prompting
Overall Accuracy: 0.6637 (148/223 correct)

Per-Category Performance:
------------------------------
Theory:
  Accuracy: 0.7222
  Correct: 26/36

Numerical:
  Accuracy: 0.2727
  Correct: 3/11

Grouping:
  Accuracy: 0.6000
  Correct: 3/5

Fill In Blank:
  Accuracy: 0.6707
  Correct: 55/82

True False:
  Accuracy: 0.6854
  Correct: 61/89



### In-Context Learning

In [8]:
for MODEL_NAME in ["gpt-35-turbo"]:
    y_pred = []
    y_true = []
    
    for row in tqdm(df_test.iterrows(), total=len(df_test)):
        time.sleep(1)
        question = row[1]["Question"]
        choice_a = row[1]["OptionA"]
        choice_b = row[1]["OptionB"]
        choice_c = row[1]["OptionC"]
        choice_d = row[1]["OptionD"]
        label = row[1]["Answer"].strip().upper()  # Clean the true label
        
        if MODEL_NAME == "gpt-35-turbo":
            if row[1]["true_false"]:
                prediction = prompt_chatgpt_with_backoff(icl_prompt_tf.format(
                    question, choice_a, choice_b))
            else:
                prediction = prompt_chatgpt_with_backoff(icl_prompt.format(
                    question, choice_a, choice_b, choice_c, choice_d))
        
        # Clean and extract the predicted answer letter more carefully
        pred_letter = prediction.strip().upper()
        # If the prediction contains more than just the letter, take first word
        if len(pred_letter) > 1:
            pred_letter = pred_letter.split()[0]
        
        # Validate prediction format
        if pred_letter not in ['A', 'B', 'C', 'D']:
            print(f"Warning: Invalid prediction format: '{prediction}' for question: {question}")
            pred_letter = pred_letter[0] # take first letter
        
        y_pred.append(pred_letter)
        y_true.append(label)

        """
        # Print debugging info for all predictions
        if pred_letter != label:
            print("\nIncorrect Prediction:")
            print(f"Question: {question}")
            print(f"Choices:")
            print(f"A) {choice_a}")
            print(f"B) {choice_b}")
            if not row[1]["true_false"]:
                print(f"C) {choice_c}")
                print(f"D) {choice_d}")
            print(f"Raw model response: '{prediction}'")
            print(f"Processed prediction: '{pred_letter}'")
            print(f"Correct answer: '{label}'")
            print("=========================================================")
        """

    # Calculate overall accuracy
    total_questions = len(y_true)
    correct_answers = sum(1 for pred, true in zip(y_pred, y_true) if pred == true)
    overall_accuracy = correct_answers / total_questions

    # Print detailed matching information
    print("\nDetailed Answer Matching:")
    print("------------------------")
    mismatches = [(i, pred, true) for i, (pred, true) in enumerate(zip(y_pred, y_true)) if pred != true]
    print(f"Found {len(mismatches)} mismatches out of {total_questions} questions")
    
    # Calculate per-category accuracies
    categories = ['theory', 'numerical', 'grouping', 'fill_in_blank', 'true_false']
    category_results = {}
    
    for category in categories:
        # Get questions belonging to this category
        category_indices = df_test[df_test[category] == 1].index
        
        if len(category_indices) > 0:
            category_correct = sum(1 for i in category_indices if y_pred[i] == y_true[i])
            category_accuracy = category_correct / len(category_indices)
            category_results[category] = {
                'accuracy': category_accuracy,
                'correct': category_correct,
                'total': len(category_indices)
            }

    # Print results
    print(f"\nResults for {MODEL_NAME}: ICL Prompting")
    print("=" * 50)
    print(f"Overall Accuracy: {overall_accuracy:.4f} ({correct_answers}/{total_questions} correct)")
    print("\nPer-Category Performance:")
    print("-" * 30)
    
    for category, results in category_results.items():
        print(f"{category.replace('_', ' ').title()}:")
        print(f"  Accuracy: {results['accuracy']:.4f}")
        print(f"  Correct: {results['correct']}/{results['total']}")
        print()

    # Save results to file
    with open("results/LLM_results.txt", "a") as f:
        f.write(f"\nResults for {MODEL_NAME}: ICL Prompting\n")
        f.write("=" * 50 + "\n")
        f.write(f"Overall Accuracy: {overall_accuracy:.4f} ({correct_answers}/{total_questions} correct)\n")
        f.write("\nPer-Category Performance:\n")
        f.write("-" * 30 + "\n")
        
        for category, results in category_results.items():
            f.write(f"{category.replace('_', ' ').title()}:\n")
            f.write(f"  Accuracy: {results['accuracy']:.4f}\n")
            f.write(f"  Correct: {results['correct']}/{results['total']}\n\n")
        f.write("\n")

  0%|          | 0/223 [00:00<?, ?it/s]

  2%|▏         | 4/223 [03:04<3:12:40, 52.79s/it]

  2%|▏         | 5/223 [04:05<3:22:03, 55.61s/it]

  3%|▎         | 7/223 [06:06<3:30:13, 58.39s/it]

 15%|█▍        | 33/223 [23:43<1:04:27, 20.35s/it]

...

 95%|█████████▌| 212/223 [2:32:36<10:33, 57.60s/it]

gies. The countries have identical and homothetic preferences. In this setting, when trade opens:


 96%|█████████▌| 213/223 [2:33:37<09:46, 58.61s/it]

100%|██████████| 223/223 [2:39:16<00:00, 42.85s/it]


Detailed Answer Matching:
------------------------
Found 65 mismatches out of 223 questions

Results for gpt-35-turbo: ICL Prompting
Overall Accuracy: 0.7085 (158/223 correct)

Per-Category Performance:
------------------------------
Theory:
  Accuracy: 0.6944
  Correct: 25/36

Numerical:
  Accuracy: 0.5455
  Correct: 6/11

Grouping:
  Accuracy: 0.6000
  Correct: 3/5

Fill In Blank:
  Accuracy: 0.6707
  Correct: 55/82

True False:
  Accuracy: 0.7753
  Correct: 69/89

